# Data Analysis Extra - Playlists

Tras los análisis anteriores, de los que no obtube lso resultados que esperaba, decidí hacer un análisis de última hora con las playslist que crearon los usuarios durante la pandemia. Tal vez aquí haya indicadores más claros de un cambio en el tipo de música antes de la pandemia y durante.



El proceso a realizar es buscar las playlists que tengan en el nombre alguna palabra relacionada con la pandemia (me he limitado al español) y extraer todas sus canciones para luego hacer un proceso de análisis de Features y Genre.

In [269]:
# Ver : https://towardsdatascience.com/what-music-do-people-listen-to-during-the-coronavirus-a-report-using-data-science-1a2035d12430
# Leer: https://blog.chartmetric.com/covid-19-effect-on-the-global-music-business-part-1-genre/
# https://gist.github.com/ilias1111/e503bbab0a98c20377686cc75ffad451


## Preparación Previa

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from datetime import datetime
import time
import altair as alt

# Meter credenciales de Spotify
passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Cargamos funciones de anteriores Notebooks simplificadas para extraer features

In [ ]:
##### Función para sacar las features de canciones que me interesan desde la id
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  valence = features[0]['valence']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  id = features[0]['id']

  track = [name, album, artist, release_date, length, popularity,
           acousticness, danceability, energy, instrumentalness,
           liveness, loudness, speechiness, valence, tempo, time_signature, id]
  return track


##### Loop para sacar features de todas las canciones de una lista, devuelve un dataframe

def extract_songs(list_toextract):
    import time
    tracks = []
    for i in range(len(list_toextract)):
        time.sleep(.5)
        track = getTrackFeatures(list_toextract[i])
        tracks.append(track)

    # Metemos la info en Dataframe
    data_features = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date',
                                                 'length', 'popularity','acousticness', 'danceability', 'energy',
                                                 'instrumentalness', 'liveness', 'loudness',
                                                 'speechiness', 'valence','tempo', 'time_signature', 'id'])

    data_features_final = data_features [['name','artist','album','release_date','length', 'popularity',
                                                 'acousticness', 'danceability', 'energy',
                                                 'instrumentalness', 'liveness', 'loudness',
                                                 'speechiness', 'valence','tempo', 'time_signature', 'id']]

    data_features_final = data_features_final.rename(columns = {'id':'spotify_id'})

    # Finalmente meto el año de Release date (lanzamiento de la canción) en un nueva columna que me va a vernir en uno de los análisis
    data_features_final['release_date_year'] = pd.to_datetime(data_features_final['release_date'])
    data_features_final['release_date_year'] = pd.DatetimeIndex(data_features_final['release_date']).year
    print(data_features_final.shape)
    data_features_final.head(4)

    #Normalizamos features
    features_to_normalize = ['length', 'popularity', 'loudness', 'tempo', 'speechiness']

    data_features_final[features_to_normalize] = data_features_final[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

    return data_features_final
    



# Pregunta 4: ¿Qué tipo de música contienen las playlists que se han creado para el tiempo de pandemia?, ¿han cambiado?

Lo primero que vamos a hacer es buscar playlists que se hayan creado "para" el coronavirus. Para eso, nos vamos a fijar en sus títulos: si contienen alguna palabra de las siguientes, nos las quedamos: "covid", "pandemia", "cuarentena",'covid19', "coronavirus", 'fernando simón'.

Posteriormente haremos el proceso de extracción de todo: playlists, canciones y features de canción. Del total de canciones(hay muchas) de las playlist del coronavirus nos quedaremos solo con las más repetidas y en dos lotes, uno peuqeño y otro más grande. La idea es comparar estos dos lotes con el Top100 del 2019, que es una playlist con las canciones más escuchadas durante del 2019.

In [34]:
# Vemos como está montada la estructura de información en la API de Spotify, para buscar una palabra en los nombres de las playlists
print(sp.search('q=fernando simón', type='playlist'))
# ["covid","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=q%3Dfernando+sim%C3%B3n&type=playlist&offset=0&limit=10', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.co

Creamos un script ya completo para buscar las palabras y sacar solo el total de listas que contiene esta palabra en alguna parte de su información

In [35]:
for word in ["fernando simón"]:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['items'])

[{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1/tracks', 'total': 271}, 'type': 'playlist', 'uri': 'spotify:playlist:01bawF2DM0YzefGbC54Ua1'}

Hacemos una extracción masiva de dichas playlist

## Extraer playlists del coronavirus

In [36]:
# Tarda alrededor de 1 minuto en completarse
# Documentación: https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/
# ['items']["tracks"]['items'][0]["added_at"]

# Función para extrar determinados features de la playlist
def batch_proccess(x,lista):
    for i in x['playlists']['items']:
        lista.append({"name" : i['name'], "total":i["tracks"]['total'], 'id':i['id'], "uri":i["uri"], "URL":i["tracks"]['href']})

# Sacar un listado de las playlist, en bloques de 50, que es el máximo que te permite la API
Time1 = datetime.now()

list_of_playlists = []
for term in ["covid","pandemia","cuarentena",'covid19', "coronavirus", 'fernando simón']:
    for i in range(0,2000, 50):
        try:
            init_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
            batch_proccess(init_data,list_of_playlists)
        except:
            print("Error")
Time2 = datetime.now()

print("Tiempo ejecución:", Time2 -Time1)

# Meto las playlists en un dataframe y echo un ojo
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)
print('Número de playlists:', playlists.count())
playlists.head(20)

Tiempo ejecución: 0:00:50.738712
Número de playlists: name     5584
total    5584
id       5584
uri      5584
URL      5584
dtype: int64


,name,total,id,uri,URL
0,quarantine Vibes | Covid 19 |,45,6zjC569rXoYGOLwPUt37WW,spotify:playlist:6zjC569rXoYGOLwPUt37WW,https://api.spotify.com/v1/playlists/6zjC569rX...
1,Calma,151,37i9dQZF1DWY5LGZYBBHHz,spotify:playlist:37i9dQZF1DWY5LGZYBBHHz,https://api.spotify.com/v1/playlists/37i9dQZF1...
2,COVID-19 Quarantine Party,29,1Tn6OrkJhOYO5u6y16erqB,spotify:playlist:1Tn6OrkJhOYO5u6y16erqB,https://api.spotify.com/v1/playlists/1Tn6OrkJh...
3,Canciones Mas Virales que el Coronavirus 🦠🎶,106,4Y2h9TZNZersxSYeXujCNu,spotify:playlist:4Y2h9TZNZersxSYeXujCNu,https://api.spotify.com/v1/playlists/4Y2h9TZNZ...
4,Drunksouls COVID19 quarantine discovery: music...,49,64KUBwKGkUEqIU08HDhJ94,spotify:playlist:64KUBwKGkUEqIU08HDhJ94,https://api.spotify.com/v1/playlists/64KUBwKGk...
5,Plenas que no pudimos bailar por COVID,81,2U4J1vAVC8DvV3fPfrgXVy,spotify:playlist:2U4J1vAVC8DvV3fPfrgXVy,https://api.spotify.com/v1/playlists/2U4J1vAVC...
6,chill beats to quarantine to,124,5bPe4ZISvKWU5yTzsSO9lX,spotify:playlist:5bPe4ZISvKWU5yTzsSO9lX,https://api.spotify.com/v1/playlists/5bPe4ZISv...
7,Quarantine covid - 19,100,7qJzyN6atsM49NOWnqGbEK,spotify:playlist:7qJzyN6atsM49NOWnqGbEK,https://api.spotify.com/v1/playlists/7qJzyN6at...
8,MÁS PERREO MENOS COVID,99,1RYr3ngXNoPVtl7blr9Kaf,spotify:playlist:1RYr3ngXNoPVtl7blr9Kaf,https://api.spotify.com/v1/playlists/1RYr3ngXN...
9,CORONAVIRUS QUARANTINE PARTY COVID-19 2020,55,3r9u5bXzAYEvc0xBKkSnAx,spotify:playlist:3r9u5bXzAYEvc0xBKkSnAx,https://api.spotify.com/v1/playlists/3r9u5bXzA...


## Extraer canciones de playlists del coronavirus

In [43]:
# Número de playlists de las que extraer canciones
playlists_extract = playlists[0:200]

In [44]:
# IMPORTANTE, Tarda alrededor de 30min en sacarlo todo, por eso ejecutamos antes una muestra sólo

#FALTA JUNTAR ARTISTAS CON NOMBRES Y VER CÓMO LO JUNTAMOS EN AL DTAFRAME FINAL
# Sacar las canciones de las listas para tenerlas todas juntas
# Tarda bastante tiempo, alrededor de XX

# Para sacar canciones
Time1 = datetime.now()
songs = []
for uri in playlists_extract['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    songs.append(k['track']["id"])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(len(songs))

Tiempo ejecución: 0:01:46.659796
37304


Hacemos un ranking de canciones más repetidas

In [45]:
#Vamos a hacer un de las canciones más repetidas
from collections import Counter
data_playlist_songs = pd.DataFrame.from_dict(Counter(songs), orient='index').reset_index().rename(columns={"index":"spotify_id", 0:"count"})
data_playlist_songs = data_playlist_songs.sort_values(by=['count'], ascending=False)
data_playlist_songs

,spotify_id,count
196,2xLMifQCjDGFmkHkpNLD9h,31
197,6I9VzXrHxO9rA9A5euc8Ak,25
6028,7ytR5pFWmSjzHJIeQkgog4,25
251,2DEZmgHKAvm41k4J3R2E9Y,24
6045,0nbXyq5TXYPCO7pr3N8S4I,24
...,...,...
10508,73thBYypIRDkDz9Fw1kgCN,1
10507,7Clnn5BpU4Ylxdz5VjrgIs,1
10506,4qFbhzOXaH98zqsmIu86o3,1
10505,0I4JJx7tSGwTqdkl1ufKLR,1


Por curiosidad, podemos echar un vistazo a cuáles son las top15 de la cuarentena. Hay que tener en cuenta de que el listado de playlist era de caracter mundial.

In [81]:
#Seleccionamos los 15 primeros
list_toextract_exitos = data_playlist_songs["spotify_id"].tolist()
list_toextract_exitos = list_toextract_exitos[0:16]
print(len(list_toextract_exitos))

# Quitar Nones, hay uno suelto
list_toextract_exitos = list(filter(None, list_toextract_exitos)) 
print(len(list_toextract_exitos))

# Extraer canciones
top_songs_pandemia = extract_songs(list_toextract_exitos)

16
15
(15, 18)


Podemos ver datos que ya sabíamos, como que Bad Bunny lo petó, pero también vemos otros curiosos como la aparición de antiguos éxitos como "U Can't Touch This" o "Don't Stand So Close To Me", que estñan relacionados con al distancia social

In [94]:
top_songs_pandemia

(15, 18)

Vamos a seguir con nuestro análisis haciendo paquetes con los top100 y top10000 

In [91]:
# Cogemos las 100 canciones que salen más veces y luego las 10000 que salen más veces para extraer sus features
data_playlist_songs_100 = data_playlist_songs[0:100]
data_playlist_songs_100 = data_playlist_songs_100["spotify_id"].tolist()
data_playlist_songs_100 = list(filter(None, data_playlist_songs_100)) 

data_playlist_songs_10000 = data_playlist_songs[0:10000]
data_playlist_songs_10000 = data_playlist_songs_10000["spotify_id"].tolist()
data_playlist_songs_10000 = list(filter(None, data_playlist_songs_10000)) 
len(data_playlist_songs_10000)

10000

## Extracción Features de canciones de Top100 y Top10000 de las playlists de la pandemia

Sacamos las features de cada canción y metemos la información en un dataframe

In [97]:
# Ejecutamos funciones de extraer canciones y normalizar de PAQUETE 1: 100
# IMPORTANTE!!! Tiempo de ejecución:
Time1 = datetime.now()

data_features_songs100 = extract_songs(data_playlist_songs_100)

Time2 = datetime.now()

# Exportamos
data_features_songs100.to_csv("data_ana_playlist_songs100.csv", sep = ',')

print("Tiempo ejecución extracción:", Time2 -Time1)
data_features_songs100.shape

(99, 18)
Tiempo ejecución extracción 100: 0:00:56.730358


(99, 18)

Echamos un vistazo a las medias, que en ste caso no podemos ponderar porque no tenemos los Streams totales

In [102]:
data_features_songs100.mean()

length                  0.447076
popularity              0.711795
acousticness            0.227604
danceability            0.747626
energy                  0.656859
instrumentalness        0.044862
liveness                0.166479
loudness                0.778376
speechiness             0.247791
valence                 0.561017
tempo                   0.429339
time_signature          4.030303
release_date_year    2017.656566
dtype: float64


In [ ]:
#Ejecutamos funciones de extraer canciones y normalizar de PAQUETE 2: 10000
# IMPORTANTE!!! Tiempo de ejecución:
Time1= datetime.now()

data_features_songs10000 = extract_songs(data_playlist_songs_10000)
data_features_songs10000 = normalize_features(data_features_songs10000)

Time2= datetime.now()

# Ha tardado casi 1 hora en extraerse así que lo guardamos CSV
data_features_songs10000.to_csv("data_ana_playlist_songs10000.csv", sep = ',')

print("Tiempo ejecución extracción 100:", Time2 -Time1)

## Extracción songs y features de playlist Top2019

Sacamos canciones y features del TopTrack 2019 de Spotify, que es una lista concreta de Spotify.

In [104]:
# Toptrack2019
songs_top19 = []
uri = 'spotify:playlist:37i9dQZF1DWTqOoGdpVCf0'
lenght = sp.playlist_tracks(uri)['total']
for i in range(0,lenght, 50):
    init_data = sp.playlist_tracks(uri, limit=50, offset=i)
    try:
        for k in init_data['items']:
            songs_top19.append(k['track']["id"])
    except:
        pass

#Ejecuto función    
Time1 = datetime.now()

data_features_songs100_2019 = extract_songs(songs_top19)

Time2 = datetime.now()

# Ha tardado casi 1 hora en extraerse así que lo guardamos CSV
data_features_songs100_2019.to_csv("data_ana_playlist_songs100_2019.csv", sep = ',')

print("Tiempo ejecución extracción top2019:", Time2 -Time1)
print(data_features_songs100_2019.shape)
data_features_songs100_2019.head(2)

(100, 18)
Tiempo ejecución extracción top2019: 0:00:57.331803
(100, 18)


,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
0,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,0.280950,1.000000,0.751,0.501,0.405,0.00,0.105,0.749490,0.006459,0.446,0.306367,4,7qEHsqek33rTcFNT9PFqLf,2019
1,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",2019-03-29,0.330765,0.977528,0.328,0.701,0.425,0.13,0.100,0.300611,0.770601,0.562,0.505711,4,2Fxmhks0bxGSBdJ92vM42m,2019


## Análisis de playlists

In [268]:
# Cargamos dataframes guardados en extracción anterior
data_ana_playlist_songs100 = pd.read_csv("data_ana_playlist_songs100.csv", sep = ',')
data_ana_playlist_songs100 = data_ana_playlist_songs100.drop(columns=['Unnamed: 0', 'time_signature', 'release_date'])

data_ana_playlist_songs100_2019 = pd.read_csv("data_ana_playlist_songs100_2019.csv", sep = ',')
data_ana_playlist_songs100_2019 = data_ana_playlist_songs100_2019.drop(columns=['Unnamed: 0', 'time_signature', 'release_date','release_date_year'])

# data_ana_playlist_songs10000 = pd.read_csv("data_ana_playlist_songs10000.csv", sep = ',')
# data_ana_playlist_songs10000 = data_ana_playlist_songs10000.drop(columns=['Unnamed: 0', 'time_signature', 'release_date','release_date_year'])


Calculamos las medias y las exportamos para hacer el análisis visual desde Tableau

In [265]:
#Calculamos medias 
data_ana_playlist_means_songs100 = data_ana_playlist_songs100.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop100_pandemia"})
data_ana_playlist_means_songs10000 = data_ana_playlist_songs10000.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop10000_pandemia"})
data_ana_playlist_means_songs100_2019 = data_ana_playlist_songs100_2019.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop100_2019"})

#Hacemos merge
data_ana_playlist_means_all = pd.merge(data_ana_playlist_means_songs100, data_ana_playlist_means_songs100_2019, on='features')

#Exportamos para analizar en Tableau
data_ana_playlist_means_all.to_csv("tableau_graph_and_analysis/data_ana_playlist_songs_totableau.csv", sep = ',')

,features,mediaTop100_pandemia,mediaTop100_2019
0,length,0.447076,0.347558
1,popularity,0.711795,0.824607
2,acousticness,0.227604,0.249406
3,danceability,0.747626,0.704750
4,energy,0.656859,0.635390
5,instrumentalness,0.044862,0.006340
6,liveness,0.166479,0.143828
7,loudness,0.778376,0.700486
8,speechiness,0.247791,0.236539
9,valence,0.561017,0.493917
